# Hello, FABRIC: Create your first FABRIC slice

## Configure the Environment

Set the environment variable that will be used by this notebook. If you are using the FABRIC, JupyterHub some of the environment will be automatically configured for you.  You will only need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  If you do not have a bastion username and private key, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

In [1]:
 import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']=<username>
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'id_rsa_fabric'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Setup the Experiment

#### Import the FABRIC API

In [2]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

#### Create the FABRIC Proxies

The FABRIC API is used via proxy objects that manage connections to the control framework.  

#### (Optional) Query Available Resources

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [3]:
try:
    available_resources = fablib.get_available_resources()
    print(f"Available Resources: {available_resources}")
    available_resources.draw()
except Exception as e:
    print(f"Error: {e}")

Available Resources: Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
TACC        10  314/320  2536/2560  116370/116400  633/635                 2/2                             3/4                            16/16               4/4               6/6
UTAH        10  302/320  2488/2560  116250/116400  632/635                 2/2                             1/4                            16/16               4/4               5/5
WASH         6  188/192  1520/1536  60580/60600    379/381                 2/2                             2/2                            10/10               2/2               3/3
MICH         6  188/192  1520/1536  60580/60600    379/38

## Create the Experiment Slice

#### Configure the Experiment Parameters



In [138]:
slice_name = 'MySlice1'
site = 'MAX'
node1_name = 'Node1'
node2_name = 'Node2'
image = 'default_ubuntu_20'
cores = 2
ram = 8
disk = 10
node1_nic_name = 'NIC1'
node2_nic_name = 'NIC2'
network_name = 'NET1'

### Create Slice

<img src="./figs/SingleNode.png" width="20%"><br>

Create a single node with basic compute capabilities. The submit function will block until the node is ready and will display a progress bar.


In [140]:
try:
    #Create Slice
    slice1 = fablib.new_slice(slice_name)

    # Add node
    node1 = slice1.add_node(name=node1_name, site=site)
    node1.set_capacities(cores=cores, ram=ram, disk=disk)
    node1.set_image(image)
    iface1 = node1.add_component(model='NIC_ConnectX_5', name=node1_nic_name).get_interfaces()[0]
    
    # Add node
    node2 = slice1.add_node(name=node2_name, site=site)
    node2.set_capacities(cores=cores, ram=ram, disk=disk)
    node2.set_image(image)
    iface2 = node2.add_component(model='NIC_ConnectX_5', name=node2_nic_name).get_interfaces()[0]
    #iface2.set_vlan(vlan='1000')
    
    # Network
    net1 = slice1.add_l2network(name=network_name, interfaces=[iface1, iface2])
    
    #Submit Slice Request
    slice1.submit()
    
    slice_id = slice1.get_slice_id()
except Exception as e:
    print(f"{e}")

Waiting for slice ................... Slice state: StableOK
Waiting for ssh in slice .. ssh successful
Running post boot config ... Done!


### Query Slices

You can get a list of all your slices from the slice manager. If this is your first slice, it should return only one slice.
 

In [141]:
try:
    slices = fablib.get_slices()
    for slice in slices:
        print(f"{slice.get_name()}, {slice.get_slice_id()}, {slice.get_state()}")
except Exception as e:
    print(f"Get Slices Fail: {e}")

MySlice1, c5233d8c-e24d-4b68-91d1-2c337a1ce390, StableOK


### Print the Node's Attributes

Each node in the slice has a set of get functions that return the node's attributes.

In [142]:
try:
    slice1 = fablib.get_slice(slice_id=slice_id)
    
    #print(f"notices = {slice.get_notices()}")
    #print(f"errors  = {slice.get_error_messages()}")
    
    for node in slice1.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print(f"   Image             : {node.get_image()}")
        print(f"   Image Type        : {node.get_image_type()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   Error Message     : {node.get_error_message()}")
        print(f"   Components        : {node.get_components()}")
        print(f"   Interfaces        : {node.get_interfaces()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print()    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Node:
   Name              : Node1
   Cores             : 2
   RAM               : 8
   Disk              : 10
   Image             : default_ubuntu_20
   Image Type        : qcow2
   Host              : max-w4.fabric-testbed.net
   Site              : MAX
   Management IP     : 63.239.135.99
   Reservation ID    : 34bd975b-571c-4735-b15f-b820ad33e5e5
   Reservation State : Active
   Error Message     : 
   Components        : [<fabrictestbed_extensions.fablib.component.Component object at 0x7f00fc0328e0>]
   Interfaces        : [<fabrictestbed_extensions.fablib.interface.Interface object at 0x7f00fc169100>, <fabrictestbed_extensions.fablib.interface.Interface object at 0x7f00fc05b040>]
   SSH Command       : ssh -i /home/fabric/.ssh/id_rsa -J minawm_0041350787@bastion-1.fabric-testbed.net ubuntu@63.239.135.99

Node:
   Name              : Node2
   Cores             : 2
   RAM               : 8
   Disk              : 10
   Image             : default_ubuntu_20
   Image Type        : qc

In [164]:
ip_n1 = "192.168.1.10"
try:
    n1 = slice1.get_node(node1_name)
    iface1 = n1.get_interface(network_name=network_name)
    
    print(f"iface: {iface1.get_name()}")
    
    iface1.set_ip(ip=ip_n1, cidr="24")

    print(f"ifname: {iface1.get_os_interface()}")
    print(f"mac: {iface1.get_mac()}")
    stdout, stderr = n1.execute(f'sudo ip addr list {iface1.get_os_interface()}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

iface: Node1-NIC1-p1
ifname: ens7
mac: 04:3f:72:fa:75:60
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 04:3f:72:fa:75:60 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.10/24 scope global ens7
       valid_lft forever preferred_lft forever




In [165]:
ip_n2 = "192.168.1.11"
try:
    n2 = slice1.get_node(node2_name)
    iface1 = n2.get_interface(network_name=network_name)
    
    iface1.set_ip(ip=ip_n2, cidr="24")

    print(f"ifname: {iface1.get_os_interface()}")
    print(f"mac: {iface1.get_mac()}")
    stdout, stderr = n2.execute(f'sudo ip addr list {iface1.get_os_interface()}')
    print (f"stdout: {stdout}")
    print (f"stderr: {stderr}")
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

ifname: ens7
mac: 04:3f:72:fa:78:00
stdout: 3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 04:3f:72:fa:78:00 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.11/24 scope global ens7
       valid_lft forever preferred_lft forever

stderr: 


In [143]:
n1 = slice1.get_node(name="Node1")
n2 = slice1.get_node(name="Node2")

## Let's do a quick ping experiment. We'll let each node ping the other.

In [170]:
print(n1.execute("ping -c 5 " + ip_n2)[0])

PING 192.168.1.11 (192.168.1.11) 56(84) bytes of data.
64 bytes from 192.168.1.11: icmp_seq=1 ttl=64 time=0.638 ms
64 bytes from 192.168.1.11: icmp_seq=2 ttl=64 time=0.076 ms
64 bytes from 192.168.1.11: icmp_seq=3 ttl=64 time=0.057 ms
64 bytes from 192.168.1.11: icmp_seq=4 ttl=64 time=0.075 ms
64 bytes from 192.168.1.11: icmp_seq=5 ttl=64 time=0.095 ms

--- 192.168.1.11 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4089ms
rtt min/avg/max/mdev = 0.057/0.188/0.638/0.225 ms



In [171]:
print(n2.execute("ping -c 5 " + ip_n1)[0])

PING 192.168.1.10 (192.168.1.10) 56(84) bytes of data.
64 bytes from 192.168.1.10: icmp_seq=1 ttl=64 time=0.121 ms
64 bytes from 192.168.1.10: icmp_seq=2 ttl=64 time=0.067 ms
64 bytes from 192.168.1.10: icmp_seq=3 ttl=64 time=0.061 ms
64 bytes from 192.168.1.10: icmp_seq=4 ttl=64 time=0.070 ms
64 bytes from 192.168.1.10: icmp_seq=5 ttl=64 time=0.131 ms

--- 192.168.1.10 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4086ms
rtt min/avg/max/mdev = 0.061/0.090/0.131/0.029 ms



## Ping experiment complete. Both nodes can see each other.

## Now let's start a kubernetes cluster on the two nodes. First node is the master. Second node is the worker.

We follow the instructions that we have here: https://github.com/apache/openwhisk-deploy-kube/blob/master/docs/k8s-diy-ubuntu.md

In [172]:
print(n1.execute("sudo apt update")[0])

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Hit:2 https://packages.cloud.google.com/apt kubernetes-xenial InRelease
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists...
Building dependency tree...
Reading state information...
64 packages can be upgraded. Run 'apt list --upgradable' to see them.



In [173]:
print(n1.execute("sudo apt install -y docker.io")[0])

Reading package lists...
Building dependency tree...
Reading state information...
docker.io is already the newest version (20.10.7-0ubuntu5~20.04.2).
0 upgraded, 0 newly installed, 0 to remove and 64 not upgraded.



In [174]:
print(n1.execute("sudo apt install -y apt-transport-https curl")[0])

Reading package lists...
Building dependency tree...
Reading state information...
curl is already the newest version (7.68.0-1ubuntu2.7).
apt-transport-https is already the newest version (2.0.6).
0 upgraded, 0 newly installed, 0 to remove and 64 not upgraded.



In [175]:
print(n1.execute("curl -s https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -")[0])

OK



In [150]:
print(n1.execute("cat <<EOF | sudo tee /etc/apt/sources.list.d/kubernetes.list\ndeb https://apt.kubernetes.io/ kubernetes-xenial main\nEOF\n")[0])

deb https://apt.kubernetes.io/ kubernetes-xenial main



In [151]:
print(n1.execute("cat /etc/apt/sources.list.d/kubernetes.list")[0])

deb https://apt.kubernetes.io/ kubernetes-xenial main



In [152]:
print(n1.execute("sudo apt update")[0])

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:5 http://security.ubuntu.com/ubuntu focal-security InRelease
Get:3 https://packages.cloud.google.com/apt kubernetes-xenial InRelease [9383 B]
Get:6 https://packages.cloud.google.com/apt kubernetes-xenial/main amd64 Packages [54.7 kB]
Fetched 64.1 kB in 2s (29.0 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
64 packages can be upgraded. Run 'apt list --upgradable' to see them.



In [153]:
print(n1.execute("sudo apt-get install -y kubelet kubeadm kubectl")[0])

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  conntrack cri-tools ebtables kubernetes-cni socat
Suggested packages:
  nftables
The following NEW packages will be installed:
  conntrack cri-tools ebtables kubeadm kubectl kubelet kubernetes-cni socat
0 upgraded, 8 newly installed, 0 to remove and 64 not upgraded.
Need to get 77.7 MB of archives.
After this operation, 335 MB of additional disk space will be used.
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 conntrack amd64 1:1.4.5-2 [30.3 kB]
Get:1 https://packages.cloud.google.com/apt kubernetes-xenial/main amd64 cri-tools amd64 1.23.0-00 [15.3 MB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 ebtables amd64 2.0.11-3build1 [80.3 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 socat amd64 1.7.3.3-2 [323 kB]
Get:2 https://packages.cloud.google.com/apt kubernetes-xenial/main amd64 ku

In [154]:
print(n1.execute("sudo apt-mark hold kubelet kubeadm kubectl")[0])

kubelet set on hold.
kubeadm set on hold.
kubectl set on hold.



In [155]:
print(n1.execute("sudo swapoff -a")[0])

In [156]:
print(n1.execute('cat <<EOF | sudo tee /etc/docker/daemon.json\n{\n\\"exec-opts\\": [\\"native.cgroupdriver=systemd\\"]\n}\nEOF\n')[0])

{
"exec-opts": ["native.cgroupdriver=systemd"]
}



In [157]:
print(n1.execute("sudo systemctl enable docker"))

('', '')


In [158]:
print(n1.execute("sudo systemctl daemon-reload"))

('', '')


In [159]:
print(n1.execute("sudo systemctl restart docker"))

('', '')


## The setup part is complete. Now we start running the kubernetes cluster.

In [131]:
n1.execute("yes | sudo kubeadm reset")

('[reset] Reading configuration from the cluster...\n[reset] FYI: You can look at this config file with \'kubectl -n kube-system get cm kubeadm-config -o yaml\'\n[reset] WARNING: Changes made to this host by \'kubeadm init\' or \'kubeadm join\' will be reverted.\n[reset] Are you sure you want to proceed? [y/N]: [preflight] Running pre-flight checks\n[reset] Stopping the kubelet service\n[reset] Unmounting mounted directories in "/var/lib/kubelet"\n[reset] Deleting contents of config directories: [/etc/kubernetes/manifests /etc/kubernetes/pki]\n[reset] Deleting files: [/etc/kubernetes/admin.conf /etc/kubernetes/kubelet.conf /etc/kubernetes/bootstrap-kubelet.conf /etc/kubernetes/controller-manager.conf /etc/kubernetes/scheduler.conf]\n[reset] Deleting contents of stateful directories: [/var/lib/etcd /var/lib/kubelet /var/lib/dockershim /var/run/kubernetes /var/lib/cni]\n\nThe reset process does not clean CNI configuration. To do so, you must remove /etc/cni/net.d\n\nThe reset process doe

In [179]:
print(n1.execute("sudo kubeadm init --pod-network-cidr=192.168.0.0/16 --apiserver-advertise-address=" + ip_n1)[0])

[init] Using Kubernetes version: v1.23.5
[preflight] Running pre-flight checks
[preflight] Pulling images required for setting up a Kubernetes cluster
[preflight] This might take a minute or two, depending on the speed of your internet connection
[preflight] You can also perform this action in beforehand using 'kubeadm config images pull'
[certs] Using certificateDir folder "/etc/kubernetes/pki"
[certs] Generating "ca" certificate and key
[certs] Generating "apiserver" certificate and key
[certs] apiserver serving cert is signed for DNS names [34bd975b-571c-4735-b15f-b820ad33e5e5-node1 kubernetes kubernetes.default kubernetes.default.svc kubernetes.default.svc.cluster.local] and IPs [10.96.0.1 192.168.1.10]
[certs] Generating "apiserver-kubelet-client" certificate and key
[certs] Generating "front-proxy-ca" certificate and key
[certs] Generating "front-proxy-client" certificate and key
[certs] Generating "etcd/ca" certificate and key
[certs] Generating "etcd/server" certificate and key

## Initialization successful. We need to save the join command somewhere, because we will need to use it later at the client.

In [180]:
print(n1.execute("mkdir -p $HOME/.kube")[0])

In [181]:
print(n1.execute("sudo cp -i /etc/kubernetes/admin.conf $HOME/.kube/config")[0])

In [182]:
print(n1.execute("sudo chown $(id -u):$(id -g) $HOME/.kube/config")[0])

In [183]:
print(n1.execute("kubectl apply -f https://docs.projectcalico.org/manifests/calico.yaml")[0])

configmap/calico-config created
customresourcedefinition.apiextensions.k8s.io/bgpconfigurations.crd.projectcalico.org created
customresourcedefinition.apiextensions.k8s.io/bgppeers.crd.projectcalico.org created
customresourcedefinition.apiextensions.k8s.io/blockaffinities.crd.projectcalico.org created
customresourcedefinition.apiextensions.k8s.io/caliconodestatuses.crd.projectcalico.org created
customresourcedefinition.apiextensions.k8s.io/clusterinformations.crd.projectcalico.org created
customresourcedefinition.apiextensions.k8s.io/felixconfigurations.crd.projectcalico.org created
customresourcedefinition.apiextensions.k8s.io/globalnetworkpolicies.crd.projectcalico.org created
customresourcedefinition.apiextensions.k8s.io/globalnetworksets.crd.projectcalico.org created
customresourcedefinition.apiextensions.k8s.io/hostendpoints.crd.projectcalico.org created
customresourcedefinition.apiextensions.k8s.io/ipamblocks.crd.projectcalico.org created
customresourcedefinition.apiextensions.k8

In [184]:
print(n1.execute("kubectl get nodes")[0])

NAME                                         STATUS     ROLES                  AGE   VERSION
34bd975b-571c-4735-b15f-b820ad33e5e5-node1   NotReady   control-plane,master   26s   v1.23.5



## Node status is "Ready".

# Client side setup.

In [185]:
print(n2.execute("sudo apt update")[0])

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1347 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [234 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [9796 B]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [831 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted Translation-en [118 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 c-n-f Metadata [532 B]
Get:12 http://security.ubuntu.com/ubuntu focal-security/univer

In [186]:
print(n2.execute("sudo apt install -y docker.io")[0])

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  bridge-utils containerd dns-root-data dnsmasq-base libidn11 pigz runc
  ubuntu-fan
Suggested packages:
  ifupdown aufs-tools cgroupfs-mount | cgroup-lite debootstrap docker-doc
  rinse zfs-fuse | zfsutils
The following NEW packages will be installed:
  bridge-utils containerd dns-root-data dnsmasq-base docker.io libidn11 pigz
  runc ubuntu-fan
0 upgraded, 9 newly installed, 0 to remove and 64 not upgraded.
Need to get 74.5 MB of archives.
After this operation, 361 MB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 pigz amd64 2.4-1 [57.4 kB]
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 bridge-utils amd64 1.6-2ubuntu1 [30.5 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/main amd64 runc amd64 1.0.1-0ubuntu2~20.04.1 [4155 kB]
Get:4 http://nova.cloud

In [187]:
print(n2.execute("sudo apt install -y apt-transport-https curl")[0])

Reading package lists...
Building dependency tree...
Reading state information...
curl is already the newest version (7.68.0-1ubuntu2.7).
curl set to manually installed.
The following NEW packages will be installed:
  apt-transport-https
0 upgraded, 1 newly installed, 0 to remove and 64 not upgraded.
Need to get 4680 B of archives.
After this operation, 162 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/universe amd64 apt-transport-https all 2.0.6 [4680 B]
Fetched 4680 B in 0s (28.6 kB/s)
Selecting previously unselected package apt-transport-https.
(Reading database ... 63931 files and directories currently installed.)
Preparing to unpack .../apt-transport-https_2.0.6_all.deb ...
Unpacking apt-transport-https (2.0.6) ...
Setting up apt-transport-https (2.0.6) ...



In [188]:
print(n2.execute("curl -s https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -")[0])

OK



In [189]:
print(n2.execute("cat <<EOF | sudo tee /etc/apt/sources.list.d/kubernetes.list\ndeb https://apt.kubernetes.io/ kubernetes-xenial main\nEOF\n")[0])

deb https://apt.kubernetes.io/ kubernetes-xenial main



In [190]:
print(n2.execute("cat /etc/apt/sources.list.d/kubernetes.list")[0])

deb https://apt.kubernetes.io/ kubernetes-xenial main



In [191]:
print(n2.execute("sudo apt update")[0])

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease
Get:3 https://packages.cloud.google.com/apt kubernetes-xenial InRelease [9383 B]
Get:6 https://packages.cloud.google.com/apt kubernetes-xenial/main amd64 Packages [54.7 kB]
Fetched 64.1 kB in 1s (122 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
64 packages can be upgraded. Run 'apt list --upgradable' to see them.



In [192]:
print(n2.execute("sudo apt install -y kubelet kubeadm kubectl")[0])

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  conntrack cri-tools ebtables kubernetes-cni socat
Suggested packages:
  nftables
The following NEW packages will be installed:
  conntrack cri-tools ebtables kubeadm kubectl kubelet kubernetes-cni socat
0 upgraded, 8 newly installed, 0 to remove and 64 not upgraded.
Need to get 77.7 MB of archives.
After this operation, 335 MB of additional disk space will be used.
Get:1 https://packages.cloud.google.com/apt kubernetes-xenial/main amd64 cri-tools amd64 1.23.0-00 [15.3 MB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 conntrack amd64 1:1.4.5-2 [30.3 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 ebtables amd64 2.0.11-3build1 [80.3 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 socat amd64 1.7.3.3-2 [323 kB]
Get:2 https://packages.cloud.google.com/apt kubernetes-xenial/main amd64 ku

In [193]:
print(n2.execute("sudo apt-mark hold kubelet kubeadm kubectl")[0])

kubelet set on hold.
kubeadm set on hold.
kubectl set on hold.



In [194]:
print(n2.execute("sudo swapoff -a")[0])

In [195]:
print(n2.execute('cat <<EOF | sudo tee /etc/docker/daemon.json\n{\n\\"exec-opts\\": [\\"native.cgroupdriver=systemd\\"]\n}\nEOF\n')[0])

{
"exec-opts": ["native.cgroupdriver=systemd"]
}



In [196]:
print(n2.execute("sudo systemctl enable docker"))

('', '')


In [197]:
print(n2.execute("sudo systemctl daemon-reload"))

('', '')


In [198]:
print(n2.execute("sudo systemctl restart docker"))

('', '')


## The setup part is complete. Now we need to do the join command. We do it like this:

As a note. When we do "kubeadm init" in the server, it prints the join command for us. What we do is that we copy it and paste it in the client. And don't forget to add sudo.

In [206]:
print(n2.execute("yes | sudo kubeadm reset")[0])

[reset] WARNING: Changes made to this host by 'kubeadm init' or 'kubeadm join' will be reverted.
[reset] Are you sure you want to proceed? [y/N]: [preflight] Running pre-flight checks
[reset] No etcd config found. Assuming external etcd
[reset] Please, manually reset etcd to prevent further issues
[reset] Stopping the kubelet service
[reset] Unmounting mounted directories in "/var/lib/kubelet"
[reset] Deleting contents of config directories: [/etc/kubernetes/manifests /etc/kubernetes/pki]
[reset] Deleting files: [/etc/kubernetes/admin.conf /etc/kubernetes/kubelet.conf /etc/kubernetes/bootstrap-kubelet.conf /etc/kubernetes/controller-manager.conf /etc/kubernetes/scheduler.conf]
[reset] Deleting contents of stateful directories: [/var/lib/kubelet /var/lib/dockershim /var/run/kubernetes /var/lib/cni]

The reset process does not clean CNI configuration. To do so, you must remove /etc/cni/net.d

The reset process does not reset or clean up iptables rules or IPVS tables.
If you wish to reset

In [207]:
output = n2.execute("sudo kubeadm join 192.168.1.10:6443 --token 6lnr2t.111a83z168va4w7a --discovery-token-ca-cert-hash sha256:a473c0a0ee8641d477f324a4bfe98dcba02ae4f36fff449532aa31f2d9a20167")
print(output[0])
print(output[1])

[preflight] Running pre-flight checks
[preflight] Reading configuration from the cluster...
[preflight] FYI: You can look at this config file with 'kubectl -n kube-system get cm kubeadm-config -o yaml'
[kubelet-start] Writing kubelet configuration to file "/var/lib/kubelet/config.yaml"
[kubelet-start] Writing kubelet environment file with flags to file "/var/lib/kubelet/kubeadm-flags.env"
[kubelet-start] Starting the kubelet
[kubelet-start] Waiting for the kubelet to perform the TLS Bootstrap...

This node has joined the cluster:
* Certificate signing request was sent to apiserver and a response was received.
* The Kubelet was informed of the new secure connection details.

Run 'kubectl get nodes' on the control-plane to see this node join the cluster.


W0324 15:02:11.072721   24673 utils.go:69] The recommended value for "resolvConf" in "KubeletConfiguration" is: /run/systemd/resolve/resolv.conf; the provided value is: /run/systemd/resolve/resolv.conf



## Let's check back at the server.

In [208]:
print(n1.execute("kubectl get nodes")[0])

NAME                                         STATUS   ROLES                  AGE     VERSION
34bd975b-571c-4735-b15f-b820ad33e5e5-node1   Ready    control-plane,master   6m25s   v1.23.5
4439cb97-d6c3-497c-b864-206c5e356948-node2   Ready    <none>                 2m26s   v1.23.5



## Node has successfully joined the cluster.

# Deploying a hello world application.

## First, we pull a hello world image and create a "deployment".

In [209]:
print(n1.execute("kubectl create deployment kubernetes-bootcamp --image=gcr.io/google-samples/kubernetes-bootcamp:v1")[0])

deployment.apps/kubernetes-bootcamp created



## Now let's do some status commands.

In [210]:
print(n1.execute("kubectl get pods --all-namespaces")[0])

NAMESPACE     NAME                                                                 READY   STATUS    RESTARTS   AGE
default       kubernetes-bootcamp-65d5b99f84-s2gcb                                 1/1     Running   0          110s
kube-system   calico-kube-controllers-56fcbf9d6b-8l4vw                             1/1     Running   0          19m
kube-system   calico-node-ppv8k                                                    1/1     Running   0          19m
kube-system   calico-node-spxc2                                                    1/1     Running   1          15m
kube-system   coredns-64897985d-gg4rp                                              1/1     Running   0          19m
kube-system   coredns-64897985d-sf467                                              1/1     Running   0          19m
kube-system   etcd-34bd975b-571c-4735-b15f-b820ad33e5e5-node1                      1/1     Running   0          19m
kube-system   kube-apiserver-34bd975b-571c-4735-b15f-b820ad33e5e5-node1

## Pick the correct pod name and run the command below.

In [212]:
print(n1.execute("kubectl describe pod kubernetes-bootcamp-65d5b99f84-s2gcb")[0])

Name:         kubernetes-bootcamp-65d5b99f84-s2gcb
Namespace:    default
Priority:     0
Node:         4439cb97-d6c3-497c-b864-206c5e356948-node2/10.20.4.132
Start Time:   Thu, 24 Mar 2022 15:13:57 +0000
Labels:       app=kubernetes-bootcamp
              pod-template-hash=65d5b99f84
Annotations:  cni.projectcalico.org/containerID: 7de5129549a327f3045b06c89ec9cb11b96652b19cb99ebdc205d89808c2c0f1
              cni.projectcalico.org/podIP: 192.168.122.129/32
              cni.projectcalico.org/podIPs: 192.168.122.129/32
Status:       Running
IP:           192.168.122.129
IPs:
  IP:           192.168.122.129
Controlled By:  ReplicaSet/kubernetes-bootcamp-65d5b99f84
Containers:
  kubernetes-bootcamp:
    Container ID:   docker://7c24b3f3cfdf9ead89758ba12ffd9afa33449af87381bb59adff7a40f07d0853
    Image:          gcr.io/google-samples/kubernetes-bootcamp:v1
    Image ID:       docker-pullable://gcr.io/google-samples/kubernetes-bootcamp@sha256:0d6b8ee63bb57c5f5b6156f446b3bc3b3c143d233037f3a2

## The next thing we need to do is to create what is called a "service".

We are going to use it to expose the deployment to the outside, through a port, which is 8080. Like this:

Note that the service itself will still need to be exposed. There's another "expose" step that we need to make.

In [214]:
print(n1.execute('kubectl expose deployment/kubernetes-bootcamp --type="ClusterIP" --port 8080')[0])

service/kubernetes-bootcamp exposed



## Let's check if the service was created.

In [215]:
print(n1.execute("kubectl get service kubernetes-bootcamp")[0])

NAME                  TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
kubernetes-bootcamp   ClusterIP   10.108.6.42   <none>        8080/TCP   57s



## Finally, we need to run a port forwarding command in order to expose the service to the outside.

Modify the --address flag. Use the "CLUSTER-IP" that is output form the command above.

In [225]:
print(n1.execute("kubectl port-forward --address 10.108.6.42 service/kubernetes-bootcamp 8080:8080 > /dev/null 2>&1 &"))

('', '')


## Now our application should finally be visible. Let's test the deployment on the master machine itself.

In [227]:
print(n1.execute("curl 10.108.6.42:8080")[0])

Hello Kubernetes bootcamp! | Running on: kubernetes-bootcamp-65d5b99f84-s2gcb | v=1



## Delete Slice

Please delete your slicd when you are done with your experiment.

In [135]:
try:
    slice = fablib.get_slice(slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")